In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jubileedemo import meatspace_protocols as meat
from jubileedemo import user_functions
import numpy as np
from bayesopt import acquisitions 
import json

import cv2

import bayesopt.bayesian_optimizer as bayesian_optimizer
import bayesopt.model as model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels

In [3]:
from jubileedemo import image_processing as img

In [32]:
jub = meat.BayesianOptDemoDriver(deck_config_filepath = 'deck_config.json')

Loading deck config file from deck_config.json


In [33]:
jub.safe_z = None

In [7]:
# prime pumps
jub.prime_lines('2.A2', volume = 28)


# Get 12 points of random data

In [20]:
sample_space = user_functions.get_constrained_points(101)

In [21]:
n_init_points = 12
sampled_indices = np.random.randint(0, len(sample_space), size = n_init_points)
sampled_points = sample_space[sampled_indices]

In [22]:
rgb_vals = []
for i in range(len(sampled_points)):
    point = sampled_points[i,:]
    well = jub.next_sample_well()
    print('creating sample')
    print(point)
    print(well)
    
    RGB = user_functions.sample_point(jub, point, 5, well)
    rgb_vals.append(RGB)

creating sample
[0.37 0.49 0.14]
3.A1
Dispensing volumes:  [1.85, 2.45, 0.7000000000000001]
well image function
current active tool:  1
creating sample
[0.72 0.14 0.14]
3.A2
Dispensing volumes:  [3.5999999999999996, 0.7000000000000001, 0.7000000000000001]
well image function
current active tool:  1
creating sample
[0.35 0.61 0.04]
3.A3
Dispensing volumes:  [1.7500000000000002, 3.05, 0.2]
well image function
current active tool:  1
creating sample
[0.55 0.2  0.25]
3.A4
Dispensing volumes:  [2.75, 1.0, 1.25]
well image function
current active tool:  1
creating sample
[0.27 0.33 0.4 ]
3.B1
Dispensing volumes:  [1.35, 1.6500000000000001, 2.0]
well image function
current active tool:  1
creating sample
[0.25 0.25 0.5 ]
3.B2
Dispensing volumes:  [1.25, 1.25, 2.5]
well image function
current active tool:  1
creating sample
[0.26 0.12 0.62]
3.B3
Dispensing volumes:  [1.3, 0.6, 3.1]
well image function
current active tool:  1
creating sample
[0.17 0.22 0.61]
3.B4
Dispensing volumes:  [0.8500000

In [34]:
loss_vals = [user_functions.color_loss_calculation(target_color, user_functions.normalize_color(rgb)) for rgb in rgb_vals]

In [44]:
init_data = []
for i in range(len(sampled_points)):
    data = {}
    data['sample_id'] = str(i)
    data['RYB_point'] = list(sampled_points[i])
    data['RGB_measured'] = rgb_vals[i]

    init_data.append(data)
    

In [45]:
with open('initial_data_random_12.jsonl', 'wt') as f:
    for entry in init_data:
        f.write(json.dumps(entry) + '\n')

# BO campaign

In [40]:
target_color = (137, 243, 190) # seafoam green
target_color = user_functions.normalize_color(target_color)

In [41]:
init_data = []

with open('initial_data_24pts_combine.jsonl', 'rt') as f:
    for line in f:
        init_data.append(json.loads(line))

In [42]:
init_X = np.zeros((len(init_data),3))
init_y = np.zeros(len(init_data))

for i, entry in enumerate(init_data):
    init_X[i,:] = entry['RYB_point']
    init_y[i] = user_functions.color_loss_calculation(target_color, user_functions.normalize_color(entry['RGB_measured']))
    

In [43]:
acquisition_function = acquisitions.optimize_EI

In [44]:
initial_data = (init_X, init_y)
number_of_iterations = 12
jubilee = jub

In [ ]:
sample_volume = 5 # mL
n_points = 101

# define possible sampling grid
available_points = user_functions.get_constrained_points(n_points)
kernel = kernels.Matern(nu = 1/2)
#kernel = kernels.DotProduct()
internal_model = model.GaussianProcessModel(kernel, scale = True, alpha = 1e-5)
acq_kwargs = {'xi':0.25}
# we know we are working with a 3-variable constrained design space here so can just hard-code that
# instantiate a bayesian optimizer object
bo = bayesian_optimizer.BayesianOptimizer(None, acquisition_function, internal_model, initial_data, valid_points = available_points, acq_kwargs = acq_kwargs)

# check that we have enough sample wells for the number of iterations we want to run 

# get first set of points from model
query_point = bo.campaign_iteration(None, None)[0]

print('first query pt: ', query_point)
rgb_values_sampled = []
ryb_points_sampled = []
for i in range(number_of_iterations):
    # query point from BO model
    # get well
    print(f'Starting iteration {i}')
    well = jubilee.next_sample_well()
    # run point in real world
    print(f'Dispensing into well {well}')
    print(f'RYB values tested: {query_point}')
    new_color = user_functions.sample_point(jubilee, query_point, sample_volume, well)
    ryb_points_sampled.append(query_point)
    rgb_values_sampled.append(new_color)
    norm_rgb = user_functions.normalize_color(new_color)
    score = user_functions.color_loss_calculation(target_color, user_functions.normalize_color(norm_rgb))

    print(f'loss score: {score}')

    print(f'RGB values observed: {norm_rgb}')
    query_point = bo.campaign_iteration(query_point, score)[0]

first query pt:  [0.   0.74 0.26]
Starting iteration 0
Dispensing into well 3.A1
RYB values tested: [0.   0.74 0.26]
RYB after norm in sample func:  [0.0, 0.74, 0.26]
Dispensing volumes:  [0.0, 3.7, 1.3]
current active tool:  1
swapping
loss score: 0.7578494638241895
RGB values observed: [0.507367500218698, 0.8118171934165636, 0.5506047313762982]
Starting iteration 1
Dispensing into well 3.A2
RYB values tested: [0.   0.65 0.35]
RYB after norm in sample func:  [0.0, 0.65, 0.35000000000000003]
Dispensing volumes:  [0.0, 3.25, 1.7500000000000002]
current active tool:  1
swapping
loss score: 0.8878508471493264
RGB values observed: [0.5214456566565441, 0.8677704045289244, 0.6738699558854772]
Starting iteration 2
Dispensing into well 3.A3
RYB values tested: [0.  0.6 0.4]
RYB after norm in sample func:  [0.0, 0.6, 0.4]
Dispensing volumes:  [0.0, 3.0, 2.0]
current active tool:  1
swapping
loss score: 0.8648242112581447
RGB values observed: [0.47362401429660456, 0.8524465439458129, 0.6808762918

In [17]:
rgb_values_sampled

[[232.92976418100702, 68.76800509878903, 106.5544933078394],
 [214.89904397705544, 188.51790949649458, 56.20280433397068],
 [143.3304015296367, 108.46016571064372, 161.66449968132568],
 [199.39502868068834, 66.61924792861696, 60.53435309114085],
 [1.0916398713826367, 180.52734225621415, 219.6636073932441],
 [115.89496494582536, 115.61644359464627, 165.61147227533462],
 [178.87482472912683, 105.61516889738687, 158.9182919056724],
 [118.88986615678776, 108.74327597195666, 117.24869343530911],
 [123.64576163161249, 85.58750796685787, 121.10630975143404],
 [205.8302103250478, 60.15615041427661, 72.3176545570427],
 [114.634034416826, 121.30860420650096, 102.04079031230083],
 [86.5812619502868, 87.1338432122371, 114.8089228808158]]

In [18]:
init_data

[{'sample_id': '0',
  'RYB_point': [0.37, 0.49, 0.14],
  'RGB_measured': [101.51293817718292,
   103.28718929254302,
   172.56010197578075]},
 {'sample_id': '1',
  'RYB_point': [0.72, 0.14, 0.14],
  'RGB_measured': [133.88859145952836, 95.62651370299554, 181.11612492033143]},
 {'sample_id': '2',
  'RYB_point': [0.35000000000000003, 0.61, 0.04],
  'RGB_measured': [115.27877629063097, 136.3812619502868, 200.83671128107073]},
 {'sample_id': '3',
  'RYB_point': [0.55, 0.2, 0.25],
  'RGB_measured': [125.35436583811345, 92.0970044614404, 144.84537922243467]},
 {'sample_id': '4',
  'RYB_point': [0.27, 0.33, 0.4],
  'RGB_measured': [128.74225621414914,
   108.36583811344805,
   151.20038240917782]},
 {'sample_id': '5',
  'RYB_point': [0.25, 0.25, 0.5],
  'RGB_measured': [184.46513702995537, 166.2295729764181, 126.10325047801147]},
 {'sample_id': '6',
  'RYB_point': [0.26, 0.12, 0.62],
  'RGB_measured': [183.86271510516252,
   148.26602931803697,
   114.18891013384321]},
 {'sample_id': '7',
  '

In [28]:
for i in range(12, 24):
    point = {}
    point['sample_id'] = str(i)
    point['RYB_point'] = list(ryb_points_sampled[i-12])
    point['RGB_measured'] = rgb_values_sampled[i-12]
    init_data.append(point)

In [29]:
init_data

[{'sample_id': '0',
  'RYB_point': [0.37, 0.49, 0.14],
  'RGB_measured': [101.51293817718292,
   103.28718929254302,
   172.56010197578075]},
 {'sample_id': '1',
  'RYB_point': [0.72, 0.14, 0.14],
  'RGB_measured': [133.88859145952836, 95.62651370299554, 181.11612492033143]},
 {'sample_id': '2',
  'RYB_point': [0.35000000000000003, 0.61, 0.04],
  'RGB_measured': [115.27877629063097, 136.3812619502868, 200.83671128107073]},
 {'sample_id': '3',
  'RYB_point': [0.55, 0.2, 0.25],
  'RGB_measured': [125.35436583811345, 92.0970044614404, 144.84537922243467]},
 {'sample_id': '4',
  'RYB_point': [0.27, 0.33, 0.4],
  'RGB_measured': [128.74225621414914,
   108.36583811344805,
   151.20038240917782]},
 {'sample_id': '5',
  'RYB_point': [0.25, 0.25, 0.5],
  'RGB_measured': [184.46513702995537, 166.2295729764181, 126.10325047801147]},
 {'sample_id': '6',
  'RYB_point': [0.26, 0.12, 0.62],
  'RGB_measured': [183.86271510516252,
   148.26602931803697,
   114.18891013384321]},
 {'sample_id': '7',
  '

In [31]:
with open('initial_data_24pts_combine.jsonl', 'wt') as f:
    for entry in init_data:
        f.write(json.dumps(entry) + '\n')

In [70]:
#image = cv2.imread('sampleimage_masked_1687813525.8915021.jpg')
image = cv2.imread('sampleimage_masked_1687813298.037668.jpg')

In [71]:
from jubileedemo import image_processing

In [72]:
rgb = image_processing._get_rgb_avg(image)

swapping


In [73]:
norm_rgb = user_functions.normalize_color(rgb)

In [74]:
user_functions.color_loss_calculation(target_color, norm_rgb)

0.2617500255474996

In [75]:
norm_rgb

[0.6621086280218055, 0.19737187036334725, 0.28539823707644374]

In [76]:
target_color

[0.5333333333333333, 0.011764705882352941, 0.9882352941176471]